In [ ]:
import pandas as pd
import numpy as np

# Exploratory analysis of NBA 2K23 data
Check out the scraping script if you're curious where this data originated. Let's load it up:

In [2]:


df = _deepnote_execute_sql("""SELECT *
FROM 'nba2k23_data_final.csv'
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df

,url,scrape_date,Team,Name,Overall Rating,Nationality,Archetype,Position,Weight (kg),Jersey,...,Block,Lateral Quickness,Help Defense IQ,Pass Perception,Defensive Consistency,Offensive Rebound,Defensive Rebound,Badges,Height (cm),Height (in)
0,https://www.2kratings.com/trae-young,2023-02-17,Atlanta Hawks,Trae Young,89,['United States'],Versatile Offensive Force,['PG'],81,11,...,35,70,58,70,30,36,41,{'Agent 3': {'Badge Category': 'Shooting Badge...,185,"6'1"""
1,https://www.2kratings.com/dejounte-murray,2023-02-17,Atlanta Hawks,Dejounte Murray,85,['United States'],2-Way Playshot,['PG'],81,5,...,47,87,85,89,85,35,62,{'Ankle Braces': {'Badge Category': 'DEF/REB B...,193,"6'4"""
2,https://www.2kratings.com/clint-capela,2023-02-17,Atlanta Hawks,Clint Capela,83,['Switzerland'],Paint Beast,['C'],115,15,...,80,60,86,74,85,93,92,{'Post Move Lockdown': {'Badge Category': 'DEF...,208,"6'10"""
3,https://www.2kratings.com/john-collins,2023-02-17,Atlanta Hawks,John Collins,82,['United States'],Inside the Arc Scorer,"['PF', 'C']",102,20,...,76,57,74,38,60,45,85,{'Aerial Wizard': {'Badge Category': 'Finishin...,206,"6'9"""
4,https://www.2kratings.com/onyeka-okongwu,2023-02-17,Atlanta Hawks,Onyeka Okongwu,80,['United States'],2-Way Inside-the-Arc Scorer,"['C', 'PF']",108,17,...,80,69,83,65,65,81,78,{'Aerial Wizard': {'Badge Category': 'Finishin...,203,"6'8"""
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,https://www.2kratings.com/taj-gibson,2023-02-17,Washington Wizards,Taj Gibson,73,['United States'],Inside the Arc Scorer,['C'],104,67,...,65,57,78,62,60,58,56,{'Vice Grip': {'Badge Category': 'Playmaking B...,206,"6'9"""
486,https://www.2kratings.com/johnny-davis,2023-02-17,Washington Wizards,Johnny Davis,72,['United States'],Mid-Range Shooter,['SG'],88,1,...,37,79,68,55,60,36,38,{},196,"6'5"""
487,https://www.2kratings.com/anthony-gill,2023-02-17,Washington Wizards,Anthony Gill,72,['United States'],Interior Finisher,['PF'],104,16,...,53,50,50,58,50,53,48,{},201,"6'7"""
488,https://www.2kratings.com/isaiah-todd-2,2023-02-17,Washington Wizards,Isaiah Todd,70,['United States'],Lob Threat,['PF'],99,14,...,50,65,58,54,40,55,59,{},206,"6'9"""


First things first, I was very curious how will the average NBA player look like.

It appears as it's 200cm (about 6'6"), can mediocrely shoot from all ranges, not very strong and in general lower in the attributes that are traditionally center-oriented (e.g. playing in the post, interior defense, etc.). We will also see further verification of this observation when we will test which players are the least close to average, and centers will dominate the list.

In [ ]:
df_to_avg = pd.concat([df['Name'], df['Team'], df['Overall Rating'], df['Height (cm)'], df['Weight (kg)'], 
                       df['Years in NBA'], df.loc[:,'Close Shot':'Defensive Rebound']], axis=1)
avg_player_dict = {}
for column in df_to_avg.columns[2:]:
    avg_player_dict[column] = int(df_to_avg[column].mean().round())
avg_player = pd.DataFrame.from_dict([avg_player_dict])
avg_player

,Overall Rating,Height (cm),Weight (kg),Years in NBA,Close Shot,Mid-Range Shot,Three-Point Shot,Free Throw,Shot IQ,Offensive Consistency,...,Interior Defense,Perimeter Defense,Steal,Block,Lateral Quickness,Help Defense IQ,Pass Perception,Defensive Consistency,Offensive Rebound,Defensive Rebound
0,77,200,97,5,80,74,74,77,77,69,...,57,65,48,55,70,69,64,58,50,62


In [ ]:
diff_from_avg = pd.concat([df_to_avg['Name'], df_to_avg['Team'] , df_to_avg.iloc[:, 2:] - df_to_avg.iloc[:, 2:].mean()], axis=1)
diff_abs = diff_from_avg.iloc[:,2:].apply(pd.DataFrame.abs)
diff_from_avg['Difference from mean'] = diff_abs.mean(axis=1)

Now, we know what the mose average NBA player looks like. Let's test which real NBA players are closest to it!

Here are the most average players:

In [ ]:


df_2 = _deepnote_execute_sql("""SELECT Name, Team, "Difference from mean"
FROM diff_from_avg
ORDER BY 3
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_2

,Name,Team,Difference from mean
0,Jeremy Sochan,San Antonio Spurs,4.362755
1,Yuta Watanabe,Brooklyn Nets,4.469388
2,Naji Marshall,New Orleans Pelicans,5.191429
3,Cedi Osman,Cleveland Cavaliers,5.196020
4,Terance Mann,Los Angeles Clippers,5.521939
...,...,...,...
485,Bismack Biyombo,Phoenix Suns,18.545714
486,Boban Marjanovic,Houston Rockets,19.566633
487,Udoka Azubuike,Utah Jazz,19.611939
488,Giannis Antetokounmpo,Milwaukee Bucks,20.315408


And here are the least average players: (you can see that this list is generally center-oriented, further strengthening our previous observation that centers are a unique player-type.)

In [ ]:


df_3 = _deepnote_execute_sql("""SELECT Name, Team, "Difference from mean"
FROM diff_from_avg
ORDER BY 3 DESC
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_3

,Name,Team,Difference from mean
0,LeBron James,Los Angeles Lakers,20.579694
1,Giannis Antetokounmpo,Milwaukee Bucks,20.315408
2,Udoka Azubuike,Utah Jazz,19.611939
3,Boban Marjanovic,Houston Rockets,19.566633
4,Bismack Biyombo,Phoenix Suns,18.545714
...,...,...,...
485,Terance Mann,Los Angeles Clippers,5.521939
486,Cedi Osman,Cleveland Cavaliers,5.196020
487,Naji Marshall,New Orleans Pelicans,5.191429
488,Yuta Watanabe,Brooklyn Nets,4.469388


Fun fact: the most average distanced-from-average player is... Chimezie Metu.

Also, our own Deni Avdija is 94th-percentile in his averageness - not bad!

Now let's find the most average player, this time, partitioned by its primary position:

In [ ]:
df['Primary Position'] = df['Position'].str[0]
df['Position'] = df['Position'].apply(pd.eval)
diff_to_avg_with_pos = pd.concat([df_to_avg, df['Primary Position']], axis=1)
diff_from_avg_player_per_pos = pd.concat([diff_to_avg_with_pos['Primary Position'], diff_to_avg_with_pos.iloc[:,:2], 
    diff_to_avg_with_pos.iloc[:, 2:-1] - diff_to_avg_with_pos.iloc[:, 2:].groupby('Primary Position').transform('mean')], axis=1)
diff_per_pos_abs = diff_from_avg_player_per_pos.iloc[:,3:].apply(pd.DataFrame.abs)
diff_from_avg_player_per_pos['Difference from mean'] = diff_per_pos_abs.mean(axis=1)


Most Average players per primary position: (play with the toggle, then tap the query and tap ctrl + enter to update it)

In [13]:
PrimaryPosition = 'C'

In [14]:


df_4 = _deepnote_execute_sql("""SELECT "Primary Position", Name, Team, "Difference from mean"
FROM diff_from_avg_player_per_pos
WHERE "Primary Position" = {{PrimaryPosition}}
ORDER BY 4
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_4

,Primary Position,Name,Team,Difference from mean
0,C,James Wiseman,Detroit Pistons,5.090110
1,C,Omer Yurtseven,Miami Heat,5.757967
2,C,Naz Reid,Minnesota Timberwolves,5.831868
3,C,Isaiah Stewart,Detroit Pistons,5.870055
4,C,Jaxson Hayes,New Orleans Pelicans,5.870330
...,...,...,...,...
86,C,Bam Adebayo,Miami Heat,13.193681
87,C,DeAndre Jordan,Denver Nuggets,13.503022
88,C,Udoka Azubuike,Utah Jazz,13.750000
89,C,Joel Embiid,Philadelphia 76ers,16.624725


Least Average players per primary position

In [15]:
PrimaryPosition = 'C'

In [16]:


df_5 = _deepnote_execute_sql("""SELECT "Primary Position", Name, Team, "Difference from mean"
FROM diff_from_avg_player_per_pos
WHERE "Primary Position" = {{PrimaryPosition}}
ORDER BY 4 DESC
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_5

,Primary Position,Name,Team,Difference from mean
0,C,Nikola Jokic,Denver Nuggets,17.070330
1,C,Joel Embiid,Philadelphia 76ers,16.624725
2,C,Udoka Azubuike,Utah Jazz,13.750000
3,C,DeAndre Jordan,Denver Nuggets,13.503022
4,C,Bam Adebayo,Miami Heat,13.193681
...,...,...,...,...
86,C,Jaxson Hayes,New Orleans Pelicans,5.870330
87,C,Isaiah Stewart,Detroit Pistons,5.870055
88,C,Naz Reid,Minnesota Timberwolves,5.831868
89,C,Omer Yurtseven,Miami Heat,5.757967


Now let's find the average player, this type partitioned by team:

In [ ]:
avg_player_per_team = df_to_avg.groupby('Team').mean()
avg_player_per_team['Sum of Skills'] = avg_player_per_team.loc[:, 'Close Shot': ].sum(axis=1)
avg_player_per_team = avg_player_per_team.reset_index()

Some observations:
* Jazz are the most tall, most heavy and most slow (see below) AND the lowest average overall rating in the NBA.
* Thunder are both 3nd tallest and 2nd leanest! (trying to find the new unicorn/KD/Giannis.)
* Bucks, Suns, Celtics, Clippers are on top of most success-predicting metrics (and Spurs, Hornets, Pistons, Rockets are at the bottom).
* Grizzlies are a very unexperienced team, but still ranks fairly high in other success-predicting metrics. They are also the #1 team when it comes to hustle (by a pretty large margin!)
* Warriors are shortest on average and 2nd highest overall rating on average.
* Mavericks are 3rd lowest in average overall rating


In [ ]:


df_6 = _deepnote_execute_sql("""SELECT Team, "Overall Rating", "Height (cm)", "Weight (kg)", "Years in NBA", "Sum of Skills"
FROM avg_player_per_team
ORDER BY 6 DESC
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_6

,Team,Overall Rating,Height (cm),Weight (kg),Years in NBA,Sum of Skills
0,Boston Celtics,78.687500,201.000000,101.937500,7.125000,2445.562500
1,Golden State Warriors,78.000000,197.312500,94.437500,6.750000,2444.312500
2,Milwaukee Bucks,78.200000,199.133333,100.066667,8.200000,2440.400000
3,Orlando Magic,76.937500,202.187500,100.187500,3.687500,2435.875000
4,Phoenix Suns,78.812500,198.000000,97.625000,7.437500,2427.187500
5,Los Angeles Lakers,77.750000,199.437500,94.125000,5.750000,2425.500000
6,New York Knicks,77.266667,197.800000,98.600000,5.200000,2419.133333
7,New Orleans Pelicans,77.352941,199.470588,96.882353,5.235294,2418.470588
8,Philadelphia 76ers,77.117647,199.176471,98.235294,7.000000,2417.588235
9,Memphis Grizzlies,77.176471,199.588235,98.882353,3.647059,2407.705882


### Let's compare some more interesting metrics:  

In [ ]:
avg_player_per_team['Total Avg Speed'] = (avg_player_per_team['Speed'] + 
                                          avg_player_per_team['Acceleration'])/2
avg_player_per_team['Total Avg IQ'] = (avg_player_per_team['Shot IQ'] +
                                   avg_player_per_team['Pass IQ'] +
                                   avg_player_per_team['Help Defense IQ'])/3

Some remarks:
* Bucks, Warriors, Cavaliers are highest IQ team (Spurs, Rockets, Jazz are lowest)
* Lakers, Hornets, Knicks are fastest teams (Jazz, Celtics, Grizzlies the slowest - interesting!)
* Warriors, Lakers, 76ers are best 3pt shooters (this beautifully reveales the flaw of averages analysis - having a bunch of slightly-above-average shooters such as the case with the Lakers, doesn't translate to high quality 3pt shooting in real life) (Knicks, Hornets, Clippers are the worst on average - again pretty interesting about the Clippers!)
    * This metrics might be skewed by players who aren't really shooting 3s - scoring 25 vs 55 on 3pt shooting both translates to "not a 3pt shooter", but will skew the data significantly. - it might be interesting to normalize it (perhaps by binning!)
* Suns, Hornets, 76ers are best at drawing fouls (Pacers, Warriors, Celtics are the worst)
* Grizzlies, Kings, Knicks are most hustle (Suns, Spurs, Hornets are least)

In [ ]:


df_7 = _deepnote_execute_sql("""SELECT Team, "Total Avg IQ", "Total Avg Speed", "Draw Foul", Hustle, "Three-Point Shot"
FROM avg_player_per_team
ORDER BY 6
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_7

,Team,Total Avg IQ,Total Avg Speed,Draw Foul,Hustle,Three-Point Shot
0,New York Knicks,73.155556,75.600000,63.800000,84.933333,69.133333
1,Charlotte Hornets,71.200000,76.433333,65.733333,78.266667,69.866667
2,Los Angeles Clippers,71.791667,68.718750,58.562500,80.750000,70.875000
3,Detroit Pistons,69.490196,72.470588,59.176471,80.117647,70.882353
4,Toronto Raptors,72.549020,72.029412,61.352941,81.705882,70.941176
5,Atlanta Hawks,69.705882,72.882353,60.529412,81.588235,71.117647
6,Memphis Grizzlies,71.627451,68.323529,60.588235,86.294118,71.588235
7,Utah Jazz,67.976190,63.678571,60.785714,81.071429,72.500000
8,San Antonio Spurs,67.549020,71.823529,58.470588,78.235294,72.882353
9,Washington Wizards,70.666667,70.875000,60.000000,81.250000,73.062500


### Let's compare US players and non-US player:

In [ ]:
df['Nationality'] = df['Nationality'].apply(pd.eval)
df['Primary Nationality'] = df['Nationality'].str[0]

75% of players are from the United States, 5% from Canada, and the rest is pretty scattered

In [ ]:


df_8 = _deepnote_execute_sql("""SELECT "Primary Nationality", COUNT(1) as count_players, (COUNT(1)/490.0)*100.0 as relative_percentage
FROM df
GROUP BY 1
ORDER BY 2 DESC
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_8

,Primary Nationality,count_players,relative_percentage
0,United States,368,75.102041
1,Canada,22,4.489796
2,Australia,9,1.836735
3,France,9,1.836735
4,Nigeria,7,1.428571
5,Germany,6,1.224490
6,Serbia,5,1.020408
7,Spain,5,1.020408
8,Italy,3,0.612245
9,Slovenia,3,0.612245


It seems as the non-US player is more likely to be of a big man archetype: taller, heavier, stronger, less explosive, better interior defender but worst perimeter defender, more skillful in the post, less of a playmaker, etc.)

In [ ]:
df['is_from_us'] = df['Primary Nationality'] == 'United States'
df_to_avg['is_from_us'] = df['Primary Nationality'] == 'United States'
avg_player_per_is_from_us = df_to_avg.groupby('is_from_us').mean()
avg_player_per_is_from_us['Sum of Skills'] = avg_player_per_is_from_us.loc[:, 'Close Shot':'Defensive Rebound'].sum(axis=1)
avg_player_per_is_from_us = avg_player_per_is_from_us.reset_index()
avg_player_per_is_from_us

,is_from_us,Overall Rating,Height (cm),Weight (kg),Years in NBA,Close Shot,Mid-Range Shot,Three-Point Shot,Free Throw,Shot IQ,...,Perimeter Defense,Steal,Block,Lateral Quickness,Help Defense IQ,Pass Perception,Defensive Consistency,Offensive Rebound,Defensive Rebound,Sum of Skills
0,False,76.827869,204.106557,101.172131,5.803279,79.639344,72.434426,70.934426,75.065574,75.885246,...,60.581967,44.549180,59.049180,64.704918,70.688525,62.631148,58.852459,56.196721,66.483607,2374.991803
1,True,76.638587,198.317935,96.105978,5.350543,80.301630,74.605978,75.546196,77.706522,77.241848,...,65.896739,49.413043,53.793478,71.146739,68.516304,64.842391,57.826087,47.396739,60.002717,2395.690217


Let's continue testing this hypothesis:

We can see that 33% and 23% of non-us players are C and PF respectively, where only 13% and 18% from US players.
It's also surprising to see how much the SG position is dominated by US players. 


In [ ]:


df_9 = _deepnote_execute_sql("""SELECT is_from_us, "Primary Position", 
       (CAST(COUNT(1) AS float) / SUM(COUNT(1)) OVER(PARTITION BY is_from_us))*100 as percentage_per_is_us
FROM df
GROUP BY 1, 2
ORDER BY 1,2
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_9

,is_from_us,Primary Position,percentage_per_is_us
0,False,C,33.606556
1,False,PF,22.950819
2,False,PG,13.934426
3,False,SF,18.852459
4,False,SG,10.655738
5,True,C,13.586956
6,True,PF,18.478260
7,True,PG,18.478260
8,True,SF,21.739130
9,True,SG,27.717390


### Let's now explore the various player archetypes:

We can immediately notice that most archetype are associated with single players only

In [ ]:


df_11 = _deepnote_execute_sql("""SELECT sub.Name, sub.Team, sub."Overall Rating", sub.Archetype, 
       total_count.total_per_archetype, total_count.avg_overall_rating
FROM (
    SELECT Name, Team, Archetype, "Overall Rating",
           RANK() OVER(PARTITION BY Archetype ORDER BY "Overall Rating" DESC) as rank_per_archetype
    FROM df
    ) sub
LEFT JOIN (
    SELECT Archetype, COUNT(1) as total_per_archetype, ROUND(AVG("Overall Rating"), 2) as avg_overall_rating
    FROM df
    GROUP BY 1
    ORDER BY 2 DESC
) total_count on sub.Archetype = total_count.Archetype
WHERE rank_per_archetype = 1
ORDER BY total_count.total_per_archetype DESC
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_11

,Name,Team,Overall Rating,Archetype,total_per_archetype,avg_overall_rating
0,Karl-Anthony Towns,Minnesota Timberwolves,86,3-Level Scorer,37,77.41
1,Lauri Markkanen,Utah Jazz,86,3-Level Scorer,37,77.41
2,Cameron Johnson,Brooklyn Nets,80,Inside-Out Scorer,35,76.09
3,Bennedict Mathurin,Indiana Pacers,80,Inside-Out Scorer,35,76.09
4,Chimezie Metu,Sacramento Kings,77,Interior Finisher,25,74.04
...,...,...,...,...,...,...
103,Matthew Dellavedova,Sacramento Kings,69,2-Way Floor General,1,69.00
104,Austin Rivers,Minnesota Timberwolves,74,2-Way Inside-Out Creator,1,74.00
105,Blake Griffin,Boston Celtics,76,Slashing Five,1,76.00
106,Amir Coffey,Los Angeles Clippers,72,Finesse Finisher,1,72.00


It seems there are no significant differences in overall ratings between single-player archetype vs. multi-player archetypes:

In [7]:


df_15 = _deepnote_execute_sql("""SELECT AVG(CASE WHEN total_per_archetype = 1 THEN avg_overall_rating ELSE NULL END) as single_player_archetype_avg_rating,
AVG(CASE WHEN total_per_archetype > 1 THEN avg_overall_rating ELSE NULL END) as multi_player_archetype_avg_rating

FROM (
    SELECT Archetype, ROUND(AVG("Overall Rating"),2) as avg_overall_rating, COUNT(1) as total_per_archetype
    FROM df
    GROUP BY 1
    ORDER BY 2 DESC
) sub

""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_15

,single_player_archetype_avg_rating,multi_player_archetype_avg_rating
0,77.34375,77.190847


Lets see which players are of the lowest IQ in the game:

In [9]:


df_13 = _deepnote_execute_sql("""SELECT Name, Team, "Overall Rating", "Shot IQ", "Pass IQ", "Help Defense IQ", 
       ("Shot IQ" + "Pass IQ" + "Help Defense IQ")/3.0 AS total_iq_avg
FROM df
ORDER BY 7
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_13

,Name,Team,Overall Rating,Shot IQ,Pass IQ,Help Defense IQ,total_iq_avg
0,Montrezl Harrell,Philadelphia 76ers,76,30,55,60,48.333333
1,Udoka Azubuike,Utah Jazz,72,40,34,74,49.333333
2,Dominick Barlow,San Antonio Spurs,66,40,52,57,49.666667
3,DaQuan Jeffries,New York Knicks,70,50,53,50,51.000000
4,Chris Silva,Dallas Mavericks,73,40,54,60,51.333333
...,...,...,...,...,...,...,...
485,Stephen Curry,Golden State Warriors,96,98,97,78,91.000000
486,Ja Morant,Memphis Grizzlies,94,98,98,79,91.666667
487,Joel Embiid,Philadelphia 76ers,97,98,82,96,92.000000
488,LeBron James,Los Angeles Lakers,98,98,97,90,95.000000


Average rating per height: no significant difference really

In [73]:


df_14 = _deepnote_execute_sql("""SELECT "Height (cm)", AVG("Overall Rating"), COUNT(1) as total_players, 
       "Height (cm)" - AVG("Height (cm)") OVER() as diff_from_avg_height
FROM df
GROUP BY 1
ORDER BY 1 DESC
""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_14

,Height (cm),"avg(""Overall Rating"")",total_players,diff_from_avg_height
0,224,73.000000,1,22.055556
1,221,87.000000,1,19.055556
2,218,75.000000,3,16.055556
3,216,75.166667,6,14.055556
4,213,76.850000,20,11.055556
5,211,80.600000,30,9.055556
6,208,77.800000,35,6.055556
7,206,76.288889,45,4.055556
8,203,76.692308,52,1.055556
9,201,76.160000,50,-0.944444


In [71]:
DeepnoteChart(df_14, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","tooltip":true},"encoding":{"x":{"sort":null,"type":"quantitative","field":"Height (cm)","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"y":{"sort":null,"type":"quantitative","field":"avg(\\\\\\"Overall Rating\\\\\\")","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"sort":null,"type":"quantitative","field":"total_players","scale":{"scheme":"blues"},"aggregate":"sum"}}}]},{"layer":[{"mark":{"clip":true,"type":"trail","color":"#f58518","tooltip":true},"encoding":{"x":{"sort":null,"type":"quantitative","field":"Height (cm)","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"y":{"sort":null,"type":"quantitative","field":"total_players","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"}}},{"mark":{"size":100,"type":"point","opacity":0,"tooltip":true},"encoding":{"x":{"sort":null,"type":"quantitative","field":"Height (cm)","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"y":{"sort":null,"type":"quantitative","field":"total_players","scale":{"type":"linear","zero":false},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

It seems as draft class of 2010 has been quite disappointing from longevity perspective:

In [11]:


df_15 = _deepnote_execute_sql("""SELECT "Years in NBA", COUNT(1) as total_players, AVG("Overall Rating")
FROM df
GROUP BY 1
ORDER BY 3 DESC

""", 'SQL_DEEPNOTE_DATAFRAME_SQL')
df_15

,Years in NBA,total_players,"avg(""Overall Rating"")"
0,18,1,86.000000
1,20,2,84.000000
2,11,11,81.818182
3,16,5,81.400000
4,12,15,80.866667
5,9,24,80.791667
6,14,11,80.000000
7,7,23,79.260870
8,8,27,79.148148
9,6,34,79.058824


In [22]:
DeepnoteChart(df_15, """{"layer":[{"layer":[{"mark":{"clip":true,"type":"bar","tooltip":true},"encoding":{"x":{"sort":null,"type":"quantitative","field":"Years in NBA","scale":{"type":"linear","domainMin":-0.1},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"y":{"sort":null,"type":"quantitative","field":"total_players","scale":{"type":"linear"},"format":{"type":"default","decimals":null},"formatType":"numberFormatFromNumberType"},"color":{"sort":null,"type":"quantitative","field":"avg(\\\\\\"Overall Rating\\\\\\")","scale":{"scheme":"blueorange"}}}}]}],"title":"","config":{"legend":{}},"$schema":"https://vega.github.io/schema/vega-lite/v5.json","encoding":{}}""")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f770f9fc-15a2-479f-b902-8fcc18f53535' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>